In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Recsys2022 Challenge 

NVIDIA-Merlin team participated in [Recsys2022 challenge](http://www.recsyschallenge.com/2022/index.html) and secured 3rd position. This notebook contains the various techniques used in the solution.

### Learning Objective
In this notebook, we will learn the importance of the concepts that improved the results of the competition significantly.

- ##### Label smoothing
     When the probabilities predicted by a Classification model are higher than its accuracy we say the model is overconfident. It can be prevented by using Label smoothing. This technique basically, transforms One-hot encoded labels into smoothed labels. 
$$  \begin{array}{l}
y_{l} \ =\ ( 1\ -\ \alpha \ ) \ *\ y_{o} \ +\ ( \alpha \ /\ L)\\
\alpha :\ Label\ smoothing\\
L:\ Total\ number\ of\ label\ classes\\
y_{o} :\ One-hot\ encoded\ label\ vector
\end{array}
$$
When α is 0, we have the original one-hot encoded labels, and as α increases, we move towards smoothed labels. Read [this](https://arxiv.org/abs/1906.02629) paper to learn more about it.


- ##### Temperature Scaling
    Similar to Label Smoothing, Temperature Scaling is done to reduce the overconfidence of a model. In this, we divide the logits (inputs to the softmax function) by a scalar parameter (T) . For more information on Temperature Scaling read [this](https://arxiv.org/pdf/1706.04599.pdf) paper.
$$ softmax\ =\ \frac{e\ ^{( z_{i} \ /\ \ T)}}{\sum _{j} \ e^{( z_{j} \ /\ T)} \ } $$


- ##### Weight Tying
In this technique, we share the Embedding layer's weights which is used to convert the input to embeddings, as the softmax weights,  to convert hidden layer output to softmax layer output. This drastically reduces the number of parameters and allows the model to train better. For more information read [this](https://arxiv.org/pdf/1608.05859v3.pdf) paper.

In [2]:
import os
import cupy
import cudf
import dask_cudf
import numpy as np
import pandas as pd 

import nvtabular as nvt
from merlin.dag import ColumnSelector
from merlin.io import Dataset
from merlin.schema import Schema, Tags
from nvtabular.ops import (
    AddMetadata,
)
from merlin.schema.tags import Tags
from utils import get_drepessi_recsys2022_dataset


import tensorflow as tf
from merlin.io import Dataset
from merlin.schema import Tags
from tensorflow.keras import regularizers
from merlin.models.tf.dataset import BatchedDataset
from merlin.models.tf.utils.tf_utils import extract_topk

import merlin.models.tf as mm
from merlin.models.tf import InputBlock
from merlin.models.tf.models.base import Model
from merlin.models.tf.core.aggregation import SequenceAggregation, SequenceAggregator
from merlin.models.tf.core.transformations import (
    ItemsPredictionWeightTying,
    L2Norm,
    LogitsTemperatureScaler,
)


DATA_FOLDER = 'dressipi'
DATA_PROCESSED_FOLDER = 'dressipi_processed'

2022-08-15 05:34:34.519883: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-15 05:34:36.727601: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:06:00.0, compute capability: 7.0


## Dressipi
The [Dressipi](http://www.recsyschallenge.com/2022/dataset.html) dataset contains 1.1 M online retail sessions that resulted in a purchase. It provides details about items that were viewed in a session, the item purchased at the end of the session and numerous features of those items. The task of this competition was, given a sequence of items predict which item will be purchased at the end of a session.


<img src="images/dressipi.jpeg" alt="dressipi_dataset" style="width: 400px; float: center;">  


#### Load the Dressipi dataset

In [3]:
train, valid = get_drepessi_recsys2022_dataset(DATA_FOLDER)

## Dataset
The dataset contains:
- `session_id`, id of a session, in which a user viewed and purchased an item. 
- `item_id` which was viewed at a given `timestamp` in a session
- `purchase_id` which is the id of item bought at the end of the session. 
In addition to `timestamp`, we have `day` and `date` features for representing the chronological order in which items were viewed.

The items in the Dresspi dataset had a many features out of which we took 22 most important features, namely 
`f_3 ,f_4 ,f_5 ,f_7 ,f_17 ,f_24 ,f_30 ,f_45 ,f_46 ,f_47 ,f_50 ,f_53 ,f_55 ,f_56 ,f_58 ,f_61 ,f_63 ,f_65 ,f_68 ,f_69 ,f_72 ,f_73 `

In [4]:
train.head()

,session_id,item_id,date,f_3,f_4,f_5,f_7,f_17,f_24,f_30,...,f_61,f_63,f_65,f_68,f_69,f_72,f_73,timestamp,day,purchase_id
0,3250,11565,2020-12-24 09:34:49.357,-1.0,618.0,-1.0,394.0,-1.0,588.0,596.500000,...,462.0,861.0,-1.0,739.0,805.0,75.0,544.0,1608802489357,358,17702
1,3250,26300,2020-12-24 09:35:29.208,-1.0,618.0,-1.0,394.0,-1.0,588.0,-1.000000,...,462.0,861.0,-1.0,739.0,805.0,75.0,544.0,1608802529208,358,17702
2,3250,11114,2020-12-24 09:37:02.501,793.0,618.0,605.0,837.0,378.0,-1.0,-1.000000,...,462.0,861.0,-1.0,379.0,709.0,75.0,544.0,1608802622501,358,17702
3,3251,14819,2020-01-23 21:46:03.284,-1.0,618.0,-1.0,490.0,378.0,-1.0,-1.000000,...,898.0,861.0,521.0,393.0,780.0,75.0,544.0,1579815963284,22,11935
4,3251,27343,2020-01-23 21:58:39.644,-1.0,618.0,-1.0,490.0,378.0,-1.0,382.666667,...,462.0,861.0,521.0,351.0,780.0,75.0,544.0,1579816719644,22,11935


Let's encode the `item_id` and `purchase_id` explicitly

In [5]:
itemids = cudf.concat([train['item_id'], 
                       valid['item_id'], 
                       train['purchase_id'], 
                       valid['purchase_id']])
itemids = cudf.DataFrame(itemids).drop_duplicates().reset_index(drop=True).reset_index()
itemids.columns = ['_item_id', 'item_id_new']
itemids['_item_id'] = itemids['_item_id'] + 2

def map_itemids(df, col):
    df = df.merge(
        itemids, 
        how='left', 
        left_on=col,
        right_on='item_id_new'
    )
    df.drop([col], axis=1, inplace=True)
    if 'item_id_new' in df.columns:
        df.drop(['item_id_new'], axis=1, inplace=True)
    df = df.rename(columns={'_item_id': col})
    return(df)

train = map_itemids(train, 'item_id')
train = map_itemids(train, 'purchase_id')

valid = map_itemids(valid, 'item_id')
valid = map_itemids(valid, 'purchase_id')

## Feature Engineering with NVTabular

### Categorify

Let’s perform categorical encoding on the entire train and validation dataset.

In [6]:
%%time
item_features_names = [col for col in train.columns if 'f_' in col]
cat_features = ['session_id', ['item_id', 'purchase_id']] + item_features_names >> nvt.ops.Categorify()

features = ['timestamp','date'] + cat_features
dataset = Dataset(cudf.concat([train,valid]))
workflow0 = nvt.Workflow(features)
workflow0.fit(dataset)

# transform data
train_0 = workflow0.transform(Dataset(train))
valid_0 = workflow0.transform(Dataset(valid))

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 2.21 s, sys: 912 ms, total: 3.12 s
Wall time: 4.99 s


### GroupBy

Now let’s group the user interactions in a session. The resultant row will have a sequence of encoded items ids with which a user interacted. We will also add features like the last item the user viewed, the sequence of interactions timestamps, item id of the purchased item etc.

In [7]:
%%time
features = train_0.head().columns.tolist()

# Define Groupby Operator
to_aggregate = {
    'date': ["first", "last"],
    'item_id': ["list", 'last'],
    'timestamp': ["list"],
    'purchase_id': ['first'],
}
for name in item_features_names: 
    to_aggregate[name] = ['list']
    
groupby_features = features >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    sort_cols=["date"],
    aggs= to_aggregate,
    name_sep="_")

# Add tags needed for the t4rec models definition
item_last = groupby_features['item_id_last'] >> nvt.ops.AddMetadata(tags=[Tags.ITEM, Tags.ITEM_ID, Tags.CATEGORICAL])
item_list = groupby_features['item_id_list'] >> nvt.ops.AddMetadata(tags=[Tags.ITEM, Tags.ITEM_ID, Tags.LIST, Tags.SEQUENCE, Tags.CATEGORICAL])

# item_list = groupby_features['item_id_list'] >> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.ITEM_ID, Tags.LIST])
feature_list = groupby_features[[name+'_list' for name in item_features_names]]>> nvt.ops.AddMetadata(tags=[Tags.SEQUENCE, Tags.ITEM, Tags.LIST])
other_features = groupby_features['session_id', 'date_first', 'date_last','timestamp_list']
target_feature = groupby_features['purchase_id_first'] >> nvt.ops.AddMetadata(tags=[Tags.TARGET])

workflow1 = nvt.Workflow(item_last + item_list + feature_list + other_features + target_feature)
workflow1.fit(train_0)

# transform data
train_1 = workflow1.transform(train_0)
valid_1 = workflow1.transform(valid_0)

CPU times: user 1.33 s, sys: 1.88 s, total: 3.21 s
Wall time: 5.09 s


### Truncate and Padding for a Maximum Sequence Length

Since we want to train a sequence of interactions, the length of each interaction needs to be the same. Let’s truncate all sequences to a fixed-size of 20. The sequences which are shorter than 20 are padded by 0.

In [8]:
SESSIONS_MAX_LENGTH = 20
list_cols = [col for col in train_1.head().columns if 'list' in col and 'date' not in col]
truncated_features = list_cols >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH, pad=True) >> nvt.ops.Rename(postfix = '_seq')

final_features = [
    'session_id', 'date_first', 'date_last', 'item_id_list', 'item_id_last', 'purchase_id_first'
]

workflow2 = nvt.Workflow(final_features + truncated_features)
workflow2.fit(train_1)

# transform data
train_2 = workflow2.transform(train_1)
valid_2 = workflow2.transform(valid_1)

### Save processed data to Parquet files

Now let's save the data for training a model

In [9]:
%%time
train_ds = Dataset(train_2.to_ddf().sort_values('date_last'), schema=train_2.schema)
valid_ds = Dataset(valid_2.to_ddf().sort_values('date_last'), schema=valid_2.schema)

train_ds.to_parquet(os.path.join(DATA_PROCESSED_FOLDER, "train/"), output_files=10)
valid_ds.to_parquet(os.path.join(DATA_PROCESSED_FOLDER, "valid/"), output_files=10)

CPU times: user 21.1 s, sys: 29.2 s, total: 50.3 s
Wall time: 1min 24s


## Training - MLP

A Sequential- Multi-Layer Perceptron model with average of the sequence as final representation

#### Hyperparameters

In [10]:
SEED = 42
EPOCHS = 5
BATCH_SIZE = 512
LEARNING_RATE = 3e-1
CLIPNORM = True
DROPOUT= 0.2 
LABEL_SMOOTHING = 0.2
TEMPERATURE_SCALING = 2
OPTIMIZER_NAME = 'adam'
LOSS='CategoricalCrossentropy'

tf.keras.utils.set_random_seed(SEED)

#### Load the processed dataset

In [11]:
train = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'train/*.parquet'), shuffle=False,)
valid = Dataset(os.path.join(DATA_PROCESSED_FOLDER, 'valid/*.parquet'), shuffle=False,)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1292: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


#### Schema 
Let’s visualize the schema. From this we will select features for training the MLP model

In [12]:
train.schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id_list_seq,"(Tags.ITEM, Tags.ITEM_ID, Tags.SEQUENCE, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.item_id_purchase_id.parquet,23619.0,450.0,0.0,23619.0,item_id_purchase_id
1,f_3_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_3.parquet,7.0,16.0,0.0,7.0,f_3
2,f_4_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_4.parquet,88.0,20.0,0.0,88.0,f_4
3,f_5_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_5.parquet,13.0,16.0,0.0,13.0,f_5
4,f_7_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_7.parquet,37.0,16.0,0.0,37.0,f_7
5,f_17_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_17.parquet,6.0,16.0,0.0,6.0,f_17
6,f_24_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_24.parquet,4.0,16.0,0.0,4.0,f_24
7,f_30_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_30.parquet,401.0,46.0,0.0,401.0,f_30
8,f_45_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_45.parquet,10.0,16.0,0.0,10.0,f_45
9,f_46_list_seq,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,False,NaN,0.0,0.0,0.0,.//categories/unique.f_46.parquet,7.0,16.0,0.0,7.0,f_46


In this model, we will only use two features `item_id_last` and `item_id_list_seq` for training

In [13]:
schema_model = train.schema.select_by_name(['item_id_last', 'item_id_list_seq', 'purchase_id_first'])
schema_model

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id_last,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ITEM_ID)",int64,False,False,None,0.0,0.0,0.0,.//categories/unique.item_id_purchase_id.parquet,23619.0,450.0,0,23619,item_id_purchase_id
1,item_id_list_seq,"(Tags.ITEM, Tags.ITEM_ID, Tags.SEQUENCE, Tags....",int64,True,False,None,0.0,0.0,0.0,.//categories/unique.item_id_purchase_id.parquet,23619.0,450.0,0,23619,item_id_purchase_id
2,purchase_id_first,"(Tags.CATEGORICAL, Tags.TARGET)",int64,False,False,None,0.0,0.0,0.0,.//categories/unique.item_id_purchase_id.parquet,23619.0,450.0,0,23619,item_id_purchase_id


Let's define the cardinalities of these features. This will be used while perfoming `weight-tying`

In [14]:
schema_model['item_id_list_seq'].properties.update({})

schema_model['item_id_list_seq'].properties['embedding_sizes'] = {
    'cardinality': float(23619), 
    'dimension': float(256)
}

schema_model['item_id_list_seq'].properties['domain'] = {
    'min': 0, 
    'max': 23619
}

schema_model['item_id_last'].properties.update({})

schema_model['item_id_last'].properties['embedding_sizes'] = {
    'cardinality': float(23619), 
    'dimension': float(256)
}

schema_model['item_id_last'].properties['domain'] = {
    'min': 0, 
    'max': 23619
}

#### Build the Model
Let's create an InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [15]:
input_block = InputBlock(
        schema_model.select_by_name(['item_id_last', 'item_id_list_seq']),
        aggregation='concat',
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(embedding_dim_default=128),
)

Now we will build MLPBlock, to get the sequence of hidden representation

In [16]:
mlp_block = mm.MLPBlock(
                [64, 128],
                activation='relu',
                no_activation_last_layer=True,
                dropout=DROPOUT,
            )

The Multi-Classiffication Prediction head will have
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [17]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=TEMPERATURE_SCALING)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="purchase_id_first",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [18]:
model_mlp = Model(input_block, mlp_block, prediction_task)

2022-08-15 05:36:25.858119: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


#### Define the Optimizer

In [19]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    clipnorm=CLIPNORM
)

model_mlp.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=LABEL_SMOOTHING),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

In [20]:
model_mlp.layers[0].parallel_layers['categorical'].embedding_tables

{'item_id_last': Embedding(), 'item_id_list_seq': Embedding()}

#### Model Training

In [21]:
%%time
history = model_mlp.fit(
    train,
    validation_data=valid,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    schema=schema_model,
)

Epoch 1/5
1799/1799 [==============================] - 184s 100ms/step - loss: 8.1997 - recall_at_100: 0.2766 - mrr_at_100: 0.0513 - ndcg_at_100: 0.0930 - map_at_100: 0.0513 - precision_at_100: 0.0028 - regularization_loss: 0.0000e+00 - val_loss: 8.0944 - val_recall_at_100: 0.4383 - val_mrr_at_100: 0.1045 - val_ndcg_at_100: 0.1685 - val_map_at_100: 0.1045 - val_precision_at_100: 0.0044 - val_regularization_loss: 0.0000e+00
Epoch 2/5
1799/1799 [==============================] - 173s 96ms/step - loss: 7.8229 - recall_at_100: 0.4015 - mrr_at_100: 0.0953 - ndcg_at_100: 0.1539 - map_at_100: 0.0953 - precision_at_100: 0.0040 - regularization_loss: 0.0000e+00 - val_loss: 8.0475 - val_recall_at_100: 0.4507 - val_mrr_at_100: 0.1103 - val_ndcg_at_100: 0.1760 - val_map_at_100: 0.1103 - val_precision_at_100: 0.0045 - val_regularization_loss: 0.0000e+00
Epoch 3/5
1799/1799 [==============================] - 177s 98ms/step - loss: 7.6503 - recall_at_100: 0.4611 - mrr_at_100: 0.1162 - ndcg_at_100: 0.

### Model Evaluation

In [22]:
model_mlp.evaluate(valid_ds, batch_size=1024, return_dict=True)

78/78 [==============================] - 20s 149ms/step - loss: 8.0216 - recall_at_100: 0.4889 - mrr_at_100: 0.1332 - ndcg_at_100: 0.2017 - map_at_100: 0.1332 - precision_at_100: 0.0049 - regularization_loss: 0.0000e+00


{'loss': 8.021586418151855,
 'recall_at_100': 0.4607679545879364,
 'mrr_at_100': 0.12527573108673096,
 'ndcg_at_100': 0.19011859595775604,
 'map_at_100': 0.12527573108673096,
 'precision_at_100': 0.004607679322361946,
 'regularization_loss': 0.0}

In [23]:
def generate_recommendations(pred, df_agg, batch_size=1024, n_topk=100):
    print('Mask Predictions')
    print('Generate Top100 Recommendations')
    out_pred = []
    out_score = []
    for i in range(0, pred.shape[0]//batch_size+1):
        batch_start = (i)*batch_size
        batch_end = min((i+1)*batch_size, pred.shape[0])
        pred_tmp = pred[batch_start:batch_end]
        cp_pred = cupy.asarray(pred_tmp)
        pred_idx = cupy.argsort(-cp_pred)
        pred_idx = cupy.asnumpy(pred_idx)
        for j in range(pred_idx.shape[0]):
            topk = []
            score = []
            for k in range(n_topk):
                idx = pred_idx[j][k]
                topk.append(idx)
                score.append(pred_tmp[j][idx])
            out_pred.append(topk)
            out_score.append(score)
    
    print('Transform Top100 Recommendations')
    metadata = df_agg[['session_id', 'purchase_id_first']].to_pandas().values.tolist()
    out = []
    for i, ex in enumerate(metadata):
        session_id = ex[0]
        purchase = ex[1]
        for k in range(n_topk):
            out.append([session_id, purchase, out_pred[i][k], out_score[i][k]])

    df_rec = cudf.DataFrame(out)
    df_rec.columns = ['session_id', 'purchased', 'rec', 'score']
    return(df_rec)

def evaluate(df, add_folds=False):
    print('Model evaluation')
    df = df.drop_duplicates(['session_id', 'rec'])
    df = df.sort_values(['session_id', 'score'], ascending=False)
    df['dummy'] = 1
    df['rank'] = df[['session_id', 'dummy']].groupby('session_id').cumsum()
    df = df[df['rank']<=100]
    df.drop('dummy', inplace=True, axis=1)
    df['mrr'] = 1/df['rank']
    df.loc[df['purchased']!=df['rec'], 'mrr'] = 0
    out = {}
    mrr = df[df['purchased']==df['rec']]['mrr'].sum()/df['session_id'].drop_duplicates().shape[0]
    out['total'] = mrr
    return(out)

In [24]:
%%time
predictions = model_mlp.predict(valid, batch_size=1024, verbose=1)
ddf = valid.to_ddf()
ddf = ddf[['session_id', 'purchase_id_first']].compute()
df_rec = generate_recommendations(predictions, ddf)
val_mrr = evaluate(df_rec)['total']
print('MRR: ',val_mrr)

78/78 [==============================] - 6s 65ms/step
Mask Predictions
Generate Top100 Recommendations
Transform Top100 Recommendations
Model evaluation
MRR:  0.1252757098878555
CPU times: user 56.6 s, sys: 13.7 s, total: 1min 10s
Wall time: 1min 11s


## Training Bi-LSTM

#### Hyperparameters

In [10]:
SEED = 42
EPOCHS = 10
BATCH_SIZE = 1024 #512
LEARNING_RATE = 3e-1
CLIPNORM = True
DROPOUT= 0.2 #0.01
LABEL_SMOOTHING = 0.2
TEMPERATURE_SCALING = 2
OPTIMIZER_NAME = 'adam'
LOSS='CategoricalCrossentropy'

BI_LSTM_HIDDEN_DIM = 64
tf.keras.utils.set_random_seed(SEED)

### Model

Let's create a `BiLSTM Block`, that will have a `inputs` dictionary to send the sequence of interaction embeddings `input_sequence`

In [11]:
class BiLSTM(mm.Block):
    def __init__(self, hidden_dim= 64, **kwargs):
        self.hidden_dim = hidden_dim
        lstm = tf.keras.layers.LSTM(hidden_dim, return_sequences=False, dropout=0.05,
                                   kernel_regularizer=regularizers.l2(1e-4))
        self.lstm = tf.keras.layers.Bidirectional(lstm)
        
        super().__init__(**kwargs)
        
    def call(self, inputs, training=False, **kwargs) -> tf.Tensor:  
        interactions = inputs['input_sequence']
        sequence_representation = self.lstm(interactions)
        return sequence_representation
    
    def compute_output_shape(self, input_shape):
        input_shape = input_shape['input_sequence']
        return (input_shape[0], input_shape[1], self.hidden_dim*2)
    
    
bilstm = BiLSTM(hidden_dim=BI_LSTM_HIDDEN_DIM)

For the Bi-LSTM model, let's use only `item_id_list_seq` for training

In [12]:
schema_model = train.schema.select_by_name(['item_id_list_seq', 'purchase_id_first'])
schema_model

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,item_id_list_seq,"(Tags.SEQUENCE, Tags.ITEM, Tags.LIST, Tags.CAT...",int64,True,False,None,0.0,0.0,0.0,.//categories/unique.item_id.parquet,23497.0,449.0,0,23497,item_id
1,purchase_id_first,"(Tags.CATEGORICAL, Tags.TARGET)",int64,False,False,None,0.0,0.0,0.0,.//categories/unique.purchase_id.parquet,18908.0,397.0,0,18908,purchase_id


#### Build the Model 

Let's create a InputBlock which takes sequential features, concatenate them and return the sequence of interaction embeddings

In [13]:
inputs = InputBlock(
        schema_model,
        aggregation='concat',
        seq=True,
        max_seq_length=20,
        embedding_options=mm.EmbeddingOptions(
            embedding_dim_default=128,
            infer_embedding_sizes=True,
            infer_embedding_sizes_multiplier=2,
            infer_embeddings_ensure_dim_multiple_of_8=True
        ),
        split_sparse=True,
)

In [14]:
dense_block = mm.ParallelBlock({'input_sequence': inputs}).connect(bilstm)

A MLPBlock to get the sequence of hidden representation

In [15]:
mlp_block = mm.MLPBlock(
                [64, 32],
                activation='relu',
                no_activation_last_layer=True,
                dropout=DROPOUT,
            )

A Multi-Classiffication Prediction head which has
- Layer Normalization
- Weight Tying
- Labels as One-hot encoded vectors, used for label smoothing 
- Temperature Scaling to reduce the overconfidence of the model

In [16]:
prediction_call = L2Norm().connect(
    ItemsPredictionWeightTying(schema_model), 
    mm.LabelToOneHot(), 
    LogitsTemperatureScaler(temperature=TEMPERATURE_SCALING)
)

prediction_task = mm.MultiClassClassificationTask(
    target_name="purchase_id_first",
    pre=prediction_call,
)

Now, we connect all the blocks togther to build a model

In [17]:
model_bi_lstm = Model(dense_block, mlp_block, prediction_task)

2022-08-08 22:46:09.666455: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [20]:
#### Define the Optimizer

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    clipnorm=CLIPNORM
)

model_bi_lstm.compile(
    optimizer=optimizer,
    run_eagerly=True,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=LABEL_SMOOTHING),
    metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[100])
)

### Model Training

In [19]:
%%time
history = model_bi_lstm.fit(
    train,
    validation_data=valid,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    schema=schema_model,
    callbacks=[WandbCallback(save_model=False)],
)

2022-08-08 22:46:26.286843: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400
wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/10
900/900 [==============================] - 140s 150ms/step - loss: 17.6281 - recall_at_100: 0.1426 - mrr_at_100: 0.0181 - ndcg_at_100: 0.0399 - map_at_100: 0.0181 - precision_at_100: 0.0014 - regularization_loss: 8.6982 - val_loss: 19.8320 - val_recall_at_100: 0.1834 - val_mrr_at_100: 0.0211 - val_ndcg_at_100: 0.0497 - val_map_at_100: 0.0211 - val_precision_at_100: 0.0018 - val_regularization_loss: 10.7310
Epoch 2/10
900/900 [==============================] - 132s 146ms/step - loss: 24.2999 - recall_at_100: 0.1395 - mrr_at_100: 0.0181 - ndcg_at_100: 0.0393 - map_at_100: 0.0181 - precision_at_100: 0.0014 - regularization_loss: 15.3040 - val_loss: 22.9900 - val_recall_at_100: 0.1896 - val_mrr_at_100: 0.0189 - val_ndcg_at_100: 0.0490 - val_map_at_100: 0.0189 - val_precision_at_100: 0.0019 - val_regularization_loss: 13.9342
Epoch 3/10
900/900 [==============================] - 129s 142ms/step - loss: 24.1477 - recall_at_100: 0.1399 - mrr_at_100: 0.0177 - ndcg_at_100: 0.0391 - ma

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Model Evaluation

In [ ]:
model_bi_lstm.evaluate(valid_ds, batch_size=1024, return_dict=True)

In [20]:
%%time
predictions = model_bi_lstm.predict(valid, batch_size=1024, verbose=1)
ddf = valid.to_ddf()
ddf = ddf[['session_id', 'purchase_id_first']].compute()
df_rec = generate_recommendations(predictions, ddf)
val_mrr = evaluate(df_rec)['total']
print('MRR: ',val_mrr)

78/78 [==============================] - 5s 64ms/step
Mask Predictions
Generate Top100 Recommendations
Transform Top100 Recommendations
Model evaluation
MRR:  0.02055274926770559
CPU times: user 55.2 s, sys: 12.4 s, total: 1min 7s
Wall time: 1min 5s
